<a href="https://colab.research.google.com/github/ychoi-kr/langchain-book/blob/main/chapter5/5_2_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

In [2]:
# 코랩 보안 비밀에 OpenAI API 키를 등록한 경우
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [3]:
!pip install langchain==0.1.19 openai==1.16.2 langchain-openai==0.1.1 langchain-experimental==0.0.56 langchainhub==0.1.15

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 5-2 Agents

### Agents 사용 예

In [4]:
import langchain

langchain.verbose = True

In [5]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal"], allow_dangerous_tools=True)

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "sample_data 디렉터리에 있는 파일 목록을 알려줘"})



> Entering new AgentExecutor chain...
I can use the terminal to list the files in the sample_data directory.
Action: terminal
Action Input: ls sample_dataExecuting command:
 ls sample_data
anscombe.json
california_housing_test.csv
california_housing_train.csv
mnist_test.csv
mnist_train_small.csv
README.md


/usr/local/lib/python3.10/dist-packages/langchain_community/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


I now know the files in the sample_data directory.
Final Answer: anscombe.json, california_housing_test.csv, california_housing_train.csv, mnist_test.csv, mnist_train_small.csv, README.md

> Finished chain.


In [6]:
print(result["output"])

anscombe.json, california_housing_test.csv, california_housing_train.csv, mnist_test.csv, mnist_train_small.csv, README.md


### Tools

In [7]:
from langchain.tools import Tool

def my_super_func(param):
    return "42"

tools = [
    Tool.from_function(
        func=my_super_func,
        name="The_Answer",
        description="생명, 우주, 그리고 모든 것에 대한 궁극적인 질문의 답"
    ),
]

In [8]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "이 세계의 진리를 알려주세요"})
print(result)



> Entering new AgentExecutor chain...
I should seek the ultimate answer to the truth of this world
Action: The_Answer
Action Input: 이 세계의 진리42This answer seems to be a reference to the Hitchhiker's Guide to the Galaxy, but it may not be the ultimate truth.
Action: The_Answer
Action Input: 이 세계의 진리42I need to keep searching for the ultimate answer to the truth of this world
Action: The_Answer
Action Input: 이 세계의 진리42I now know the final answer
Final Answer: 생명, 우주, 그리고 모든 것에 대한 궁극적인 질문의 답

> Finished chain.
{'input': '이 세계의 진리를 알려주세요', 'output': '생명, 우주, 그리고 모든 것에 대한 궁극적인 질문의 답'}


In [9]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain import LLMChain

summarize_template = """아래의 글을 결론만 한 마디로 요약해 주세요.

{input}
"""
summarize_prompt = PromptTemplate(
   input_variables=["input"],
   template=summarize_template,
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

tools = [
    Tool.from_function(
        func=summarize_chain.run,
        name="Summarizer",
        description="Text summarizer"
    ),
]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [10]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

text = """다음을 요약해 주세요.

안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다. OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다. 저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다.
제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다. 하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다.
질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다."""

result = agent_executor.invoke({"input": text})
print(result)




> Entering new AgentExecutor chain...
I should use the Summarizer tool to summarize the text provided in Korean.
Action: Summarizer
Action Input: 안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다. OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다. 저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다. 제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다. 하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다. 질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다.

> Entering new LLMChain chain...
Prompt after formatting:
아래의 글을 결론만 한 마디로 요약해 주세요.

안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다. OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다. 저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다. 제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다. 하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다. 질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다.


> Finished chain.
ChatGPT는 OpenAI가 개발한 AI 언어 모델로, 다양한 주제에 대한 질문에 답하거나 대화를 나누는 것을 전문으로 합니다.I now know the final answer.

Final Answer: ChatGPT는 OpenAI가 개발한 AI 언어 모델로, 다양한 주제에 대한 질문에 답하거나 대화를 나누는 것을 전문으

### Function calling을 사용하는 OpenAI Functions Agent

In [11]:
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal"], allow_dangerous_tools=True)
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


agent_executor.invoke({"input": "sample_data 디렉터리에 있는 파일 목록을 알려줘"})




> Entering new AgentExecutor chain...

Invoking: `terminal` with `{'commands': 'ls sample_data'}`


Executing command:
 ['ls sample_data']
anscombe.json
california_housing_test.csv
california_housing_train.csv
mnist_test.csv
mnist_train_small.csv
README.md


/usr/local/lib/python3.10/dist-packages/langchain_community/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


`sample_data` 디렉터리에 있는 파일 목록은 다음과 같습니다:
- anscombe.json
- california_housing_test.csv
- california_housing_train.csv
- mnist_test.csv
- mnist_train_small.csv
- README.md

> Finished chain.


{'input': 'sample_data 디렉터리에 있는 파일 목록을 알려줘',
 'output': '`sample_data` 디렉터리에 있는 파일 목록은 다음과 같습니다:\n- anscombe.json\n- california_housing_test.csv\n- california_housing_train.csv\n- mnist_test.csv\n- mnist_train_small.csv\n- README.md'}

### 여러 도구를 한 번에 사용

In [12]:
!pip install duckduckgo_search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 24.1 MB/s eta 0:00:00


In [13]:
import langchain

langchain.debug = True
langchain.verbose = False

In [14]:
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
tools = load_tools(["ddg-search"])
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


result = agent_executor.invoke({"input": "서울과 부산의 날씨를 알려줘"})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "서울과 부산의 날씨를 알려줘"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": []
}
[chain/en

In [15]:
print(result["output"])

서울과 부산의 현재 날씨는 다음과 같습니다:

### 서울
- **기온**: 영하 6.3도
- **날씨**: 부분적으로 흐림
- **바람**: 산들 바람, 북서, 속도 4 m/s, 돌풍 11 m/s
- **습도**: 40%
- **기압**: 1004 hPa
- **자외선 지수**: 3.9 (보통)
- **가시성**: 100%

### 부산
- **기온**: 아침 최저기온 17도, 낮 최고기온 26도
- **날씨**: 대체로 맑음
- **바람**: 온화한 날씨가 이어지고 있으며, 추위로 인한 불편함은 없음

서울은 현재 매우 추운 날씨를 보이고 있으며, 부산은 비교적 온화한 날씨를 유지하고 있습니다.


### OpenAI Tools Agent

OpenAI API는 functions를 더 이상 사용하지 않고 tools를 사용합니다. tools API를 사용하면 모델에서 여러 함수를 한 번에 호출하도록 요청할 수 있으므로 일부 아키텍처에서 응답 시간을 줄일 수 있습니다. OpenAI 모델에는 tools agent를 사용하는 것이 좋습니다.

In [18]:
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
tools = load_tools(["ddg-search"])
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


result = agent_executor.invoke({"input": "서울과 부산의 날씨를 알려줘"})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "서울과 부산의 날씨를 알려줘"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": []
}
[chain/en

In [19]:
print(result["output"])

서울과 부산의 현재 날씨 정보를 알려드릴게요.

### 서울 (Seoul)
- **최저 기온**: 약 19°C (66°F)
- **최고 기온**: 약 31°C (88°F)
- **날씨**: 현재 서울의 날씨는 맑고, 바람이 약간 불고 있습니다. 

### 부산 (Busan)
- **최저 기온**: 약 5°C (41°F)
- **최고 기온**: 약 17°C (63°F)
- **날씨**: 현재 부산의 날씨는 맑고, 바람이 약간 불고 있습니다.

더 자세한 정보가 필요하시면 말씀해 주세요!


### (칼럼) Function calling을 활용한 OurputParser, Extraction, Tagging

In [20]:
import json

from langchain_openai import ChatOpenAI
from langchain.chains import create_extraction_chain

schema = {
    "properties": {
        "person_name": {"type": "string"},
        "person_height": {"type": "integer"},
        "person_hair_color": {"type": "string"},
        "dog_name": {"type": "string"},
        "dog_breed": {"type": "string"},
    },
    "required": ["person_name", "person_height"],
}
text = """
Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
Alex's dog Frosty is a labrador and likes to play hide and seek.
"""

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_extraction_chain(schema, chat)

people = chain.invoke(text)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` thatis available on ChatModels capable of tool calling.You can read more about the method here: https://python.langchain.com/docs/modules/model_io/chat/structured_output/Please follow our extraction use case documentation for more guidelineson how to do information extraction with LLMs.https://python.langchain.com/docs/use_cases/extraction/.If you notice other issues, please provide feedback here:https://github.com/langchain-ai/langchain/discussions/18154
  warn_deprecated(


[chain/start] [chain:LLMChain] Entering Chain run with input:
{
  "input": "\nAlex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.\nAlex's dog Frosty is a labrador and likes to play hide and seek.\n"
}
[llm/start] [chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Extract and save the relevant entities mentioned in the following passage together with their properties.\n\nOnly extract the properties mentioned in the 'information_extraction' function.\n\nIf a property is not present and is not required in the function parameters, do not include it in the output.\n\nPassage:\n\nAlex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.\nAlex's dog Frosty is a labrador and likes to play hide and seek."
  ]
}
[llm/end] [chain:LLMChain > llm:ChatOpenAI] [1.26s] Exiting LLM run with output:
{
  "generations": [
    [
      {
 

In [21]:
print(json.dumps(people, indent=2))

{
  "input": "\nAlex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.\nAlex's dog Frosty is a labrador and likes to play hide and seek.\n",
  "text": [
    {
      "person_name": "Alex",
      "person_height": 5,
      "person_hair_color": "blonde"
    },
    {
      "person_name": "Claudia",
      "person_height": 6,
      "person_hair_color": "brunette",
      "dog_name": "Frosty",
      "dog_breed": "labrador"
    }
  ]
}


## 요약

### (칼럼) Evaluation

In [22]:
from langchain_openai import ChatOpenAI
from langchain.evaluation import load_evaluator

chat = ChatOpenAI(model="gpt-4", temperature=0)

evaluator = load_evaluator("qa", eval_llm=chat)

result = evaluator.evaluate_strings(
    input="나는 시장에 가서 사과 10개를 샀어. 사과 2개를 이웃에게 주고, 2개를 수리공에게 주었어. 그리고 사과 5개를 더 사서 1개는 내가 먹었어. 나는 몇 개의 사과를 가지고 있었니?",
    prediction="""먼저 사과 10개로 시작했어.
이웃에게 2개, 수리공에게 2개를 나누어 주었으므로 사과가 6개가 남았어.
그런 다음 사과 5개를 더 사서 이제 사과가 11개가 되었어.
마지막으로 사과 1개를 먹었으므로 사과 10개가 남게 돼.""",
    reference="10개",
)

[chain/start] [chain:QAEvalChain] Entering Chain run with input:
{
  "query": "나는 시장에 가서 사과 10개를 샀어. 사과 2개를 이웃에게 주고, 2개를 수리공에게 주었어. 그리고 사과 5개를 더 사서 1개는 내가 먹었어. 나는 몇 개의 사과를 가지고 있었니?",
  "answer": "10개",
  "result": "먼저 사과 10개로 시작했어.\n이웃에게 2개, 수리공에게 2개를 나누어 주었으므로 사과가 6개가 남았어.\n그런 다음 사과 5개를 더 사서 이제 사과가 11개가 되었어.\n마지막으로 사과 1개를 먹었으므로 사과 10개가 남게 돼."
}
[llm/start] [chain:QAEvalChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are a teacher grading a quiz.\nYou are given a question, the student's answer, and the true answer, and are asked to score the student answer as either CORRECT or INCORRECT.\n\nExample Format:\nQUESTION: question here\nSTUDENT ANSWER: student's answer here\nTRUE ANSWER: true answer here\nGRADE: CORRECT or INCORRECT here\n\nGrade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than

In [23]:
print(result)

{'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}


위 입력은 [Chain-of-Thought Prompting | Prompt Engineering Guide](https://www.promptingguide.ai/kr/techniques/cot)에서 인용했습니다.